In [ ]:
!pip install diffusers transformers ftfy accelerate
!pip install -U controlnet_aux

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 22.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinu

In [ ]:
!pip install qrcode
!pip install pyngrok
!pip install flask_cors
!pip install requests
!pip install Flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.5 MB/s eta 0:00:00


In [ ]:
import os

qfolder = "qr"
if not os.path.exists(qfolder):
  os.mkdir(qfolder)

folder = "outputs"
if not os.path.exists(folder):
  os.mkdir(folder)

In [ ]:
import qrcode

# QRコード作成
def createQr(qrText, d):
  # 誤り訂正レベル
  #  ERROR_CORRECT_L
  #    7％以下のエラーを修正
  #  ERROR_CORRECT_M
  #    初期設定値
  #    15％以下のエラーを修正
  #  ERROR_CORRECT_Q
  #    20％以下のエラーを修正
  #  ERROR_CORRECT_H
  #    30％以下のエラーを修正
  error_correct = qrcode.constants.ERROR_CORRECT_L

  # QRコード生成時のパラメータを指定する
  # サイズは size 250x250 がよい可能性が...
  qr = qrcode.QRCode(
      version=3,
      error_correction=error_correct,
      box_size=10,
      border=2,
  )

  qr.add_data(qrText)
  qr.make()
  qrimg = qr.make_image()
  qrpath = f'qr/qr_{d}.png'
  qrimg.save(qrpath)

  return qrpath

In [ ]:
import torch
from diffusers import ControlNetModel, StableDiffusionControlNetPipeline, EulerAncestralDiscreteScheduler

# 画像生成モデル
model_path = "runwayml/stable-diffusion-v1-5"

# ControlNet の準備
# Lineart
# controlnet_main = ControlNetModel.from_pretrained(
#     "ControlNet-1-1-preview/control_v11p_sd15_lineart",
#     torch_dtype=torch.float16)

# Tile
controlnet_main = ControlNetModel.from_pretrained(
    'lllyasviel/control_v11f1e_sd15_tile',
    torch_dtype=torch.float16)
# Brightness
controlnet_brightness = ControlNetModel.from_pretrained(
    "latentcat/control_v1p_sd15_brightness",
    torch_dtype=torch.float16)

# Pipeline の準備(Main用)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
   controlnet=controlnet_main
).to("cuda")
pipe.enable_model_cpu_offload()
# Brightness
pipeB = StableDiffusionControlNetPipeline.from_pretrained(
    model_path,
    torch_dtype=torch.float16,
    controlnet=controlnet_brightness
).to("cuda")
pipeB.enable_model_cpu_offload()

# スケジューラーの設定
pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
pipeB.scheduler = EulerAncestralDiscreteScheduler.from_config(pipeB.scheduler.config)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/955 [00:00<?, ?B/s]

An error occurred while trying to fetch lllyasviel/control_v11f1e_sd15_tile: lllyasviel/control_v11f1e_sd15_tile does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


diffusion_pytorch_model.bin:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
from diffusers.utils import load_image

def genImgPre(inputImg, prompt, seed, d):
  inputImg = inputImg.resize((512, 512))

  n_prompt = "(worst quality:1.4), (low quality:1.4), (monochrome:1.3), NSFW"
  seed=torch.manual_seed(seed)

  # パラメータ調整
  cond_scale=0.6
  control_start=0
  control_end=1
  steps=25

  image = pipeB(
      prompt=prompt,
      negative_prompt=n_prompt,
      num_inference_steps=steps,
      controlnet_conditioning_scale=cond_scale,
      control_guidance_start=control_start,
      control_guidance_end=control_end,
#      strength=0.5,
      # 毎回同じseed値を使う(同じ画像を生成する)
#      generator=seed,
      image=inputImg
  ).images[0]
  image.save(f'outputs/pre_output_{d}.png')

  return image

In [ ]:
from diffusers.utils import load_image
from controlnet_aux import LineartDetector

def genImgMain(inputImg, prompt, seed, d):
  inputImg = inputImg.resize((512, 512))
  n_prompt = "(worst quality:1.4), (low quality:1.4), (monochrome:1.3), NSFW"

  # processor = LineartDetector.from_pretrained("lllyasviel/Annotators")
  # inputImg = processor(inputImg)
  seed=torch.manual_seed(seed)

  # パラメータ調整
  cond_scale=0.75
  control_start=0.1
  control_end=0.95
  steps=25

  image = pipe(
      prompt=prompt,
      negative_prompt=n_prompt,
      num_inference_steps=steps,
      controlnet_conditioning_scale=cond_scale,
     control_guidance_start=control_start,
     control_guidance_end=control_end,
#      strength=0.5,
      # 毎回同じseed値を使う(同じ画像を生成する)
#      generator=seed,
      image=inputImg
  ).images[0]
  image.save(f'outputs/main_outputL_{d}.png')

  return image

/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(


In [ ]:
# Test
# from PIL import Image
# import datetime

# now = datetime.datetime.now()
# d = now.strftime('%Y%m%d%H%M%S')

# qr_data = "https://phygitalify-qr.onrender.com/"
# prompt = "a cat in a cafe"

# 毎回同じseed値を使う(同じ画像を生成する)
# seed=torch.manual_seed(0)

# # QR画像を生成
# qrpath = createQr(qr_data)
# qrimg = Image.open(qrpath, d)
# #qrimg

# # QRコードの背景を生成
# imagePre = genImgPre(qrimg, prompt,seed, d)

# # QRコードの背景を入力にして画像生成
# image = genImgMain(imagePre, prompt, seed, d)
# image


In [ ]:
import requests
import io
import json
from flask import Flask, send_file, render_template, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok, conf
from PIL import Image
import datetime


# ngrokトークンを設定
conf.get_default().auth_token = "2iSGz2xdhjgsTOClek6eMrjpT6F_7msYimGkAoqEEbhwQcTQt"
# ngrokで公開するURL
NGROK_DOMAIN = "wallaby-more-pony.ngrok-free.app"

app = Flask(__name__)
CORS(app)

@app.route('/check')
def home():
    return "I'm ready!"

@app.route('/', methods=['POST'])
def index():

  now = datetime.datetime.now()
  d = now.strftime('%Y%m%d%H%M%S')

  json_data = request.json
  print(json_data)
  qrText = json_data["qrText"]
  prompt = json_data["imgPrompt"]
  genMode = json_data["genMode"]

  # 毎回同じseed値を使う(同じ画像を生成する)
  #seed = 0
  # seed乱数を生成
  seed = torch.randint(0, 2 ** 16, [1])
  print(seed)

  img_byte_array = io.BytesIO()

  if genMode == 0:
    # QRコード生成
    qrpath = createQr(qrText, d)
    qrimg = Image.open(qrpath).convert("RGB")
    print("genMode 0 +++++++++++++++++++++++++++")

    # 画像生成
    # QRコードの背景を生成
    imagePre = genImgPre(qrimg, prompt, seed, d)
    # QRコードの背景を入力にして画像生成
    image = genImgMain(imagePre, prompt, seed, d)

    image.save(img_byte_array, format='PNG')


  elif genMode == 1:
    print("genMode 1 +++++++++++++++++++++++++++")
    qrpath = createQr(qrText, d)
    qrimg = Image.open(qrpath).convert("RGB")
    qrimg.save(img_byte_array, format='PNG')


  print(" ========================== ")
  img_byte_array.seek(0)

	# IPFSへのファイルのピン留め（アップロード）
  url = "https://api.pinata.cloud/pinning/pinFileToIPFS"
  files = {'file': ('qrcode.png', img_byte_array, 'image/png')}
  headers = {
		'pinata_api_key': "ef4b2776bda587ee59df",
		'pinata_secret_api_key': "668688893b3ee64dcfdcc2b8ba801ecda95d916fdc307bd21f8d8351be86940c"
	}
  response = requests.post(url, files=files, headers=headers)

	# 一時ファイルを削除
  img_byte_array.close()

  getUrl = 'ipfs://' + response.json()["IpfsHash"]
  print(getUrl)

  return getUrl

if __name__ == "__main__":
  public_url = ngrok.connect(5000, hostname=NGROK_DOMAIN)
  print(f"ngrok URL: {public_url}")
  app.run(port=5000)

ngrok URL: NgrokTunnel: "https://wallaby-more-pony.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [14/Jul/2024 10:30:04] "GET /check HTTP/1.1" 200 -


{'qrText': 'test2', 'imgPrompt': 'colourful transformer fighting autobot', 'genMode': 0}
tensor([13887])
genMode 0 +++++++++++++++++++++++++++


  0%|          | 0/25 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


...

{'qrText': 'test code 1', 'imgPrompt': 'blue flower garden', 'genMode': 0}
tensor([60276])
genMode 0 +++++++++++++++++++++++++++


  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [14/Jul/2024 10:39:16] "POST / HTTP/1.1" 200 -


ipfs://QmT79Rt9645EUBeCnc6ZPbAvzSHhNueJhs9RPLyZV8zZZr
